# Based on https://til.simonwillison.net/llms/python-react-pattern

In [ ]:
import openai
import re
import httpx
import os
from dotenv import load_dotenv

_ = load_dotenv()
from openai import OpenAI

In [ ]:
# Check the OpenAI client set up
client = OpenAI()
chat_completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {
            "role": "system",
            "content": "You are a helpful assistant that translates English to Spanish.",
        },
        {
            "role": "user",
            "content": "We are learning the basics of LangChain.",
        },
    ],
    temperature=0.7,
)
chat_completion.choices[0].message.content

In [ ]:
# Create our first Agent

class SimpleReActAgent:
    def __init__(self, system=""):
        self.system = system
        self.messages = []
        if self.system:
            self.messages.append({"role": "system", "content": self.system})

    def __call__(self, message):
        self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result
    
    def execute(self):
        completation = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=self.messages,
            temperature=0, # Zero  will make it very deterministic
        )
        return completation.choices[0].message.content

In [69]:
prompt = """"
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

wikipedia:
e.g. wikipedia: Paraguay
Returns a summary from searching Wikipedia

translate:
e.g. translate: The capital of Paraguay is Asuncion
Returns a translation of the text from English to Spanish.

Example:
Question: Tell me about Paraguay
Thought: I need to find information about Paraguay using wikipedia.
Action: wikipedia: Paraguay
PAUSE

You will be called again with this:

Observation: Paraguay is a country in South America. Its capital is Asuncion.

You then output:

Answer: Paraguay is a country in South America. Its capital is Asuncion.


""".strip()

In [70]:
def wikipedia(q):
    return httpx.get("https://en.wikipedia.org/w/api.php", params={
        "action": "query",
        "list": "search",
        "srsearch": q,
        "format": "json"
    }).json()["query"]["search"][0]["snippet"]

def translate(source_lang="English", target_lang="Spanish", text=""):
    # Create a prompt for the translation
    prompt = f"Translate the following text from {source_lang} to {target_lang}: {text}"
    return prompt

# Define the actions that the agent can perform    
known_actions = {
    "wikipedia": wikipedia,
    "translate": translate
}

In [ ]:
# Manually step through the agent's process
abot = SimpleReActAgent(prompt)
result = abot("Tell me about Paraguay")
print(result)
next_prompt = "Observation: {}".format(result)
result = abot(next_prompt)
print(result)

next_prompt = "Observation: {}".format(translate(text=result or ""))
result = abot(next_prompt)
print(result)

Thought: I need to find information about Paraguay using wikipedia.
Action: wikipedia: Paraguay
PAUSE
Answer: Paraguay is a country in South America. Its capital is Asuncion.
Answer: Respuesta: Paraguay es un país en América del Sur. Su capital es Asunción.


In [64]:
abot = SimpleReActAgent(prompt)

action_re = re.compile(r'^Action: (\w+): (.*)$')   # python regular expression to selection action

def query(question, max_turns=5):
    i = 0
    bot = SimpleReActAgent(prompt)
    next_prompt = question
    while i < max_turns:
        i += 1
        result = bot(next_prompt)
        print(result)
        if result is None:
            break
        actions = [
            action_re.match(a) 
            for a in result.split('\n')
        ]
        # Filter out None values
        actions = [a for a in actions if a is not None]
        print("-"*20)
        print(actions)
        print("-"*20)
        if actions:
            # There is an action to run
            action, action_input = actions[0].groups()
            if action not in known_actions:
                raise Exception("Unknown action: {}: {}".format(action, action_input))
            print(" -- running {} {}".format(action, action_input))
            observation = known_actions[action](action_input)
            print("Observation:", observation)
            next_prompt = "Observation: {}".format(observation)
        else:
            return

In [74]:
query("What is Paraguay?. Translate the answer.")

Thought: I need to find information about Paraguay using wikipedia.
Action: wikipedia: Paraguay
PAUSE
--------------------
[<re.Match object; span=(0, 27), match='Action: wikipedia: Paraguay'>]
--------------------
 -- running wikipedia Paraguay
Observation: <span class="searchmatch">Paraguay</span>, officially the Republic of <span class="searchmatch">Paraguay</span>, is a landlocked country in South America. It is bordered by Argentina to the south and southwest, Brazil
Thought: I need to translate the information about Paraguay from English to Spanish.
Action: translate: Paraguay, officially the Republic of Paraguay, is a landlocked country in South America. It is bordered by Argentina to the south and southwest, Brazil
PAUSE
--------------------
[<re.Match object; span=(0, 170), match='Action: translate: Paraguay, officially the Repub>]
--------------------
 -- running translate Paraguay, officially the Republic of Paraguay, is a landlocked country in South America. It is bordered 

In [77]:
query("Is Bolivia bordered Paraguay?. Translate the answer.")

Thought: I need to confirm if Bolivia is bordered by Paraguay using Wikipedia.
Action: wikipedia: Bolivia
PAUSE
--------------------
[<re.Match object; span=(0, 26), match='Action: wikipedia: Bolivia'>]
--------------------
 -- running wikipedia Bolivia
Observation: <span class="searchmatch">Bolivia</span>, officially the Plurinational State of <span class="searchmatch">Bolivia</span>, is a landlocked country located in central South America. The country features diverse geography
Thought: Now that I have confirmed that Bolivia is a landlocked country in central South America, I can provide the translation of the answer.
Action: translate: Bolivia is a landlocked country located in central South America.
PAUSE
--------------------
[<re.Match object; span=(0, 84), match='Action: translate: Bolivia is a landlocked countr>]
--------------------
 -- running translate Bolivia is a landlocked country located in central South America.
Observation: Translate the following text from Bolivia is 